In [ ]:
%%time
import torch
import torch.multiprocessing as mp
at = [ torch.zeros(10) for i in range(5) ]
#print(at)

def forward(a):
    print(a)
    #q.get()#1+1
    #q.put( q.get()+2 )

# NOTE: this is required for the ``fork`` method to work
if __name__ == '__main__':
    processes = []
    queues = []
    #x = torch.zeros_like(a[0])
    #x.share_memory()
    for a in at:
        q = mp.Queue(a)
        p = mp.Process(target=forward, args=(q))
        p.start()
        processes.append(p)
    for p in processes:
        p.join()
    #print(args)
    #print(x)
    #for q in queues:
    #   print(q.get())

In [ ]:
import multiprocessing as mp
import torch

done = mp.Event()

def extractor_worker(done_queue):
    done_queue.put(torch.Tensor(10,10))
    done_queue.put(None)
    done.wait()

producers = []
done_queue = mp.Queue()
for i in range(0, 1):
    process = mp.Process(target=extractor_worker,
                         args=(done_queue,))
    process.start()
    producers.append(process)

    for p in producers:
        p.join()


In [ ]:
import numpy as np
import torch
import torch.multiprocessing as mp

torch.set_grad_enabled(False)

def target(inputs, outputs):
    x = inputs.get()
    outputs.put(x)
    outputs.join()

inputs = mp.Queue(1)
outputs = mp.JoinableQueue(1)

value = torch.tensor([1., 2.])

inputs.put(value)
proc = mp.Process(target=target, args=(inputs, outputs))
proc.start()
x = outputs.get()
outputs.task_done()
proc.join()
print(x)


In [ ]:
import torch
x = torch.Tensor(3,3)
x

In [ ]:
in_dim,out_dim=720,300
conv1d, linear = nn.Conv1d(in_dim, out_dim, 1), nn.Linear(in_dim, out_dim)

# same input tensor
tensor = torch.randn(2048,1, in_dim)
permuted_tensor = tensor.permute(0, 2, 1).clone().contiguous()

%time out_linear = linear(tensor)  # torch.Size([128, 256, 32])
%time out_conv1d = conv1d(permuted_tensor)  # torch.Size([128, 32, 256])
out_linear.shape,out_conv1d.shape

In [ ]:
import torch
import torch.nn as nn
import timeit, matplotlib.pyplot as plt


def count_params(model):
    """Count the number of parameters in a module."""
    return sum([p.numel() for p in model.parameters()])


def compare_params(linear, conv1d):
    """Compare whether two modules have identical parameters."""
    return (linear.weight.detach().numpy() == conv1d.weight.detach().numpy().squeeze()).all() and \
           (linear.bias.detach().numpy() == conv1d.bias.detach().numpy()).all()


def compare_tensors(out_linear, out_conv1d):
    """Compare whether two tensors are identical."""
    return (out_linear.detach().numpy() == out_conv1d.permute(0, 2, 1).detach().numpy()).all()
conv1d, linear = nn.Conv1d(8, 32, 1), nn.Linear(8, 32)

# same input tensor
tensor = torch.randn(128, 256, 8)
permuted_tensor = tensor.permute(0, 2, 1).clone().contiguous()

# same weights and bias
linear.weight = nn.Parameter(conv1d.weight.squeeze(2))
linear.bias = nn.Parameter(conv1d.bias)
print(compare_params(linear, conv1d))  # True

# check on the forward tensor
out_linear = linear(tensor)  # torch.Size([128, 256, 32])
out_conv1d = conv1d(permuted_tensor)  # torch.Size([128, 32, 256])
print(compare_tensors(out_linear, out_conv1d))  # False
plt.hist((out_linear.detach().numpy() - out_conv1d.permute(0, 2, 1).detach().numpy()).ravel())

In [ ]:
target = torch.randn(out_linear.shape)
permuted_target = target.permute(0, 2, 1).clone().contiguous()

loss_linear = nn.MSELoss()(target, out_linear)
loss_linear.backward()
loss_conv1d = nn.MSELoss()(permuted_target, out_conv1d)
loss_conv1d.backward()

plt.hist((linear.weight.grad.detach().numpy() - 
    conv1d.weight.grad.permute(0, 2, 1).detach().numpy()).ravel())

In [ ]:
# test execution speed on CPUs
print(timeit.timeit("_ = linear(tensor)", number=10000, setup="from __main__ import tensor, linear"))
print(timeit.timeit("_ = conv1d(permuted_tensor)", number=10000, setup="from __main__ import conv1d, permuted_tensor"))

# change everything in *.cuda(), then test speed on GPUs
%time 

In [ ]:
tensor.shape,permuted_tensor.shape

# multiprocessing

In [ ]:
import time
from multiprocess import Pool
COUNT = 10000000
start = time.perf_counter()
def countdown(n):
    while n > 0:
        n -= 1

pool = Pool(processes=2)
start = time.perf_counter()
r1 = pool.apply_async(countdown, [COUNT//2])
r2 = pool.apply_async(countdown, [COUNT//2])
pool.close()
pool.join()
print('Time used:', time.perf_counter() - start)

In [ ]:
from multiprocess import Process,Pool,Queue

def f(x):
    return x*x

if __name__ == '__main__':
    with Pool(5) as p:
        print(p.map(f, [1, 2, 3]))

In [ ]:
def f(q):
    q.put('hello world')
    
def other_function():
    q = Queue()
    p = Process(target=f, args=[q])
    p.start()
    print (q.get())
    p.join()
    
if __name__ == '__main__':
    other_function()

In [ ]:
import torch
import torch.nn as nn
import torch.multiprocessing as mp
#import multiprocess as mp

import lib.finance.train_extern_functions as extern
from lib.finance.train_extern_functions import*
"""
#def ex_forward(m:nn.Module,queue:mp.Queue):
def ex_forward(model,queue):
    #t = queue.get()*2
    #queue.put(t)
    #queue.put("hello world")
    queue.put(f"hello world:\n{model(queue.get())}")
    #queue.put( 2+queue.get() )
"""

def f(q):
   #q.put('hello world')
    
    q.put(f'hello world:\n{q.get()}')

class Model(nn.Module):
    def __init__(self,dim1,dim2):
        super().__init__()
        self.l = nn.Linear(dim1,dim2)
    def forward(self,x):
        return self.l(x)
    
def other_function():
    dim1,dim2=3,2
    x = torch.rand(dim2,dim1)
    print(f"initialized x :\n{x}\n")
    stems = [Model(dim1,dim2),Model(dim1,dim2)]
    processes = []
    queues =[]
    for s in stems:
        s.share_memory()
        q = mp.Queue()
        queues.append(q)
        q.put(x)
        p = mp.Process(target=extern.ex_forward, args=[s,q])
        #p = mp.Process(target=f, args=[q])
        p.start()
        processes.append(p)
    for p in processes:
        p.join()
        
    #x_stem = queues[0].get()
    for q in queues: #[1:]:
        if not q.empty():
            print(q.get())
        #x_stem = x_stem + q.get()
    #print(f"processed x :{x_stem}")
    
if __name__ == '__main__':
    #mp.set_start_method("fork")
    other_function()    
#dim1,dim2=3,2
#model = Model(3,2)
#x = torch.rand(dim2,dim1)
#model(x)

initialized x :
tensor([[0.1303, 0.7657, 0.5110],
        [0.4032, 0.0724, 0.8736]])

tensor([[0.1303, 0.7657, 0.5110],
        [0.4032, 0.0724, 0.8736]])
tensor([[0.1303, 0.7657, 0.5110],
        [0.4032, 0.0724, 0.8736]])


In [ ]:
import torch.multiprocessing as mp
from model import MyModel

def train(model):
    # Construct data_loader, optimizer, etc.
    for data, labels in data_loader:
        optimizer.zero_grad()
        loss_fn(model(data), labels).backward()
        optimizer.step()  # This will update the shared parameters

if __name__ == '__main__':
    num_processes = 4
    model = MyModel()
    # NOTE: this is required for the ``fork`` method to work
    model.share_memory()
    processes = []
    for rank in range(num_processes):
        p = mp.Process(target=train, args=(model,))
        p.start()
        processes.append(p)
    for p in processes:
        p.join()

In [ ]:
import time
import numpy as np
n=int(1e6)
print(n)
%time np.random.uniform(low=0, high=1, size=n)
r_indices = np.random.randint(0,n, n)
r_indices

1000000
CPU times: user 11.9 ms, sys: 4.43 ms, total: 16.4 ms
Wall time: 13.4 ms


array([761300, 996091, 133378, ..., 927159, 111212, 303312])